<a href="https://colab.research.google.com/github/alunfes/GoogleColabProjects/blob/main/Statical_Pattern_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
法則性の発見：
まずこれはmdのみに焦点を当てればいい。様々なタイムスケールでの法則が存在しうるが、標本数や確率など最低限の基準を満たす必要がある。
法則はこうなったらこうなる、というもの。こうなったらはmdの指標の組み合わせで表現される状態。こうなるは一定期間内における価格の一定以上の上下。これに標本数と確率が基準として使われる。

・ランダムに指標と条件を組み合わせる -> 標本数要件を満たすか確認 -> 期間と価格変動のパターンを組み合わせて確率基準を満たすことができるか確認する。

'''

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

#%cp -rp '/content/drive/My Drive/ta-lib/' ~/
%cd ~
!rm -rf ta-lib*
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!wget https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd '/content/drive/My Drive/ta-lib'
#%cd ta-lib
import os
#os.chdir('/content/drive/My Drive/ta-lib') # Can't use !cd in co-lab
os.chdir('ta-lib') # Can't use !cd in co-lab
#%cd ~/ta-lib
!./configure --prefix=/usr
#! '/content/drive/My Drive/ta-lib/configure' --prefix=/usr
!make
!make install
!pip install Ta-Lib
%cd /content

/root
--2024-01-23 23:01:04--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving sourceforge.net (sourceforge.net)... 172.64.150.145, 104.18.37.111, 2606:4700:4400::ac40:9691, ...
Connecting to sourceforge.net (sourceforge.net)|172.64.150.145|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/ [following]
--2024-01-23 23:01:04--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/download [following]
--2024-01-23 23:01:04--  https://sourceforge.net/projects/ta-lib/files/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz/download
Reusing existing connection to sourceforge.net:443.
HTTP request sent, aw

In [3]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import os
import numpy as np
import math
import random
import optuna
import matplotlib.pyplot as plt
import time
import datetime
import talib as ta
from collections import deque, namedtuple
from numba import jit, njit, float64, int64

In [5]:
################################################################################
########################## Config ####################################
################################################################################
ohlc_directory = '/content/drive/My Drive/MarketData/ohlc'
model_directory = '/content/drive/My Drive/Model/'

In [6]:
################################################################################
########################## MarketData ####################################
################################################################################
class MarketData:
    @classmethod
    def initialize(cls, terms:list):
        cls.terms = terms
        cls.open = []
        cls.high = []
        cls.low = []
        cls.close = []
        cls.dt = []
        cls.ma_kairi = {} #term:ma_kairi
        cls.all_index = {} #name:data
        cls.__read_data()
        cls.__calc_all_index()


    @classmethod
    def __read_data(cls):
        df = pd.read_csv(ohlc_directory+'/bybit-USDT-BTC.csv')
        cls.open = np.array(df['open'].tolist())
        cls.high = np.array(df['high'].tolist())
        cls.low = np.array(df['low'].tolist())
        cls.close = np.array(np.array(df['close'].tolist()))
        cls.dt = np.array(pd.to_datetime(df['timestamp'], unit='ms'))  # Convert timestamp to datetime

    @classmethod
    def __calc_all_index(cls):
        for term in cls.terms:
            cls.all_index['ma_kairi_'+str(term)] = cls.__calc_ma_kairi(term)
            cls.all_index['rsi_'+str(term)] = cls.__calc_rsi(term)
            cls.all_index['adx_'+str(term)] = cls.__calc_adx(term)
            cls.all_index['cci_'+str(term)] = cls.__calc_cci(term)
            cls.all_index['cmo_'+str(term)] = cls.__calc_cmo(term)
            cls.all_index['dx_'+str(term)] = cls.__calc_dx(term)
            cls.all_index['minus_di_'+str(term)] = cls.__calc_minus_di(term)
            cls.all_index['rocr_'+str(term)] = cls.__calc_rocr(term)
            cls.all_index['correl_'+str(term)] = cls.__calc_correl(term)


    @classmethod
    def __calc_ma_kairi(cls, term):
        '''
        ma = pd.Series(cls.close).rolling(window=term).mean()  # 移動平均を計算
        ma_kairi = ((pd.Series(cls.close) - ma) / ma) * 100  # 乖離率を計算
        ma_kairi[:term] = np.nan  # 最初のterm要素をnanに設定
        return ma_kairi.tolist()
        '''
        # Calculate moving average using talib's SMA (Simple Moving Average) function
        ma = ta.SMA(np.array(cls.close, dtype='f8'), timeperiod=term)
        # Calculate the deviation (kairi) from the moving average
        ma_kairi = ((np.array(cls.close, dtype='f8') - ma) / ma)
        return np.array(ma_kairi)

    @classmethod
    def __calc_rsi(cls, term):
        return ta.RSI(np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_adx(cls, term):
        return ta.ADX(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_cci(cls, term):
        return ta.CCI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term) / 1500

    @classmethod
    def __calc_cmo(cls, term):
        return ta.CMO(np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_dx(cls, term):
        return ta.DX(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_minus_di(cls, term):
        return ta.MINUS_DI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_plus_di(cls, term):
        return ta.PLUS_DI(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_rocr(cls, term):
        return ta.ROCR100(np.array(cls.close, dtype='f8'), timeperiod=term) / 100.0

    @classmethod
    def __calc_correl(cls, term):
        return ta.CORREL(np.array(cls.high, dtype='f8'), np.array(cls.low, dtype='f8'), timeperiod=term)



MarketData.initialize([5, 15, 60, 120, 240, 720])

In [7]:
################################################################################
########################## Random Index Combination Generator ########################
################################################################################
'''
ランダムに指標と条件を組み合わせる:
実際に存在する組み合わせの条件である必要がある。対象の指標が決まったら適当な地点のデータ組み合わせを対象にそれぞれの指標の値の範囲に応じてx%の上下の範囲を決める。
'''
class RandomIndexCombinationGenerator:
    def __init__(self):
        pass

    def get_randome_index_combinations(self):
        index_names = list(MarketData.all_index.keys())
        random_num = random.randint(0, len(MarketData.all_index) - 1)
        # Shuffle the index_names to get a random order
        random.shuffle(index_names)
        # Select the first 'random_num' indices
        return index_names[:random_num]



rcg = RandomIndexCombinationGenerator()
rcg.get_randome_index_combinations()

['adx_120',
 'correl_15',
 'cci_5',
 'rsi_60',
 'cmo_5',
 'correl_60',
 'rsi_720',
 'adx_60',
 'ma_kairi_720',
 'dx_5',
 'correl_240',
 'ma_kairi_120',
 'adx_240',
 'rocr_60',
 'ma_kairi_60',
 'correl_5',
 'dx_15',
 'adx_720',
 'rocr_15',
 'cci_60',
 'dx_240',
 'rsi_240',
 'correl_720',
 'correl_120',
 'rocr_120',
 'cmo_15',
 'cmo_120',
 'adx_15',
 'minus_di_5',
 'rsi_15',
 'cci_240']

In [19]:
################################################################################
########################## Index Rule Evaluator ########################
################################################################################
class IndexRuleEvaluator:
    def evaluate(self, start_ind:int, end_ind:int, index_names:list, index_kijun_max_val:list, index_kijun_min_val:list, kijun_period:int, target_pnl_ratio:float, max_drowdown_ratio:float):
        num_index_trigger_hit = 0
        num_price_change_satisfaction = 0
        hitted_price_change_conditions = []
        num_index_condition_hit_index = []
        for i in range(start_ind, end_ind + 1):
            flg = self.__check_index_conditions(i, index_names, index_kijun_max_val, index_kijun_min_val)
            if flg:
                num_index_condition_hit_index.append(i)
                hitted_price_change_conditions.append(self.__check_price_change_conditions(i, kijun_period, target_pnl_ratio, max_drowdown_ratio))
        num_up_condition_hit, num_down_condition_hit = self.__evaluate_hitted_price_change_conditions(hitted_price_change_conditions)
        if len(num_index_condition_hit_index) > 0:
            return self.__evaluation_kijun(len(num_index_condition_hit_index), num_up_condition_hit/len(num_index_condition_hit_index), num_down_condition_hit/len(num_index_condition_hit_index), kijun_period, target_pnl_ratio, max_drowdown_ratio)
        else:
            return 0

    def __evaluation_kijun(self, num_hit, up_condition_ratio, down_condition_ratio, kijun_period, target_pnl_ratio, max_drowdown_ratio):
        long_or_short_condition_prob = max(up_condition_ratio, down_condition_ratio)
        performance_eva = (target_pnl_ratio / kijun_period) / max_drowdown_ratio
        return long_or_short_condition_prob * performance_eva


    def __check_index_conditions(self, ind:int, index_names:list, index_kijun_max_val:list, index_kijun_min_val:list):
        for i in range(len(index_names)):
            if not (index_kijun_max_val[i] >= MarketData.all_index[index_names[i]][ind] and MarketData.all_index[index_names[i]][ind] >= index_kijun_min_val[i]):
                return False
        return True



    def __check_price_change_conditions(self, ind:int, kijun_period:int, target_pnl_ratio:float, max_drowdown_ratio:float):
        '''
        ①(long)一定期間内にx%以上下がる前にy%以上上がる or ②(short)一定期間内にx%以上上がる前にy%以上下がる
        どちらか片方の条件が一定上の確率で成立すればいい。
        0: no hit, 1:hit condition ①, -1:hit for condition ②
        '''
        maxp = max(MarketData.high[ind : ind + kijun_period])
        minp = min(MarketData.low[ind : ind + kijun_period])
        max_change_ratio = (maxp - MarketData.close[ind]) / MarketData.close[ind]
        min_change_ratio = (minp - MarketData.close[ind]) / MarketData.close[ind]
        maxp_ind = np.where(MarketData.high[ind : ind + kijun_period] == maxp)[0][-1]
        minp_ind = np.where(MarketData.low[ind : ind + kijun_period] == minp)[0][-1]
        #check for condition 1 (long)
        if max_change_ratio >= target_pnl_ratio and maxp_ind < minp_ind:
            #print('hit condition-1')
            #print('maxp_ind=', maxp_ind + ind, ', minp_ind=',minp_ind)
            return 1
        #check for condition 2 (short)
        if -min_change_ratio >= target_pnl_ratio and minp_ind < maxp_ind:
            return -1
        else:
            return 0


    def __evaluate_hitted_price_change_conditions(self, hitted_price_change_conditions):
        '''
        連続して同じhitが続く場合は1としてカウントする。
        *もしかしたら、より保守的に__check_index_conditionsでTrueが続く場合はすべて1としてカウントすべきかも。
        '''
        num_up_condition_hit = 0
        num_down_condition_hit = 0
        for i in range(len(hitted_price_change_conditions)-1):
            if hitted_price_change_conditions[i+1] != hitted_price_change_conditions[i]:
                if hitted_price_change_conditions[i+1] == 1:
                    num_up_condition_hit += 1
                elif hitted_price_change_conditions[i+1] == -1:
                    num_down_condition_hit += 1
        return num_up_condition_hit, num_down_condition_hit

for j in range(5):
    start_ind = 10000
    end_ind = 1500000
    ire = IndexRuleEvaluator()
    selected_index_names = rcg.get_randome_index_combinations()
    index_kijun_max_val = []
    index_kijun_min_val = []
    max_per = 90
    min_per = 10
    kijun_period = 1440
    target_pnl_ratio = 0.03
    max_drowdown_ratio = 0.02
    for i in range(len(selected_index_names)):
        index_name = selected_index_names[i]
        index_kijun_max_val.append(np.percentile(MarketData.all_index[index_name][start_ind : end_ind+1], max_per))
        index_kijun_min_val.append(np.percentile(MarketData.all_index[index_name][start_ind : end_ind+1], min_per))
    res = ire.evaluate(start_ind, end_ind, selected_index_names, index_kijun_max_val, index_kijun_min_val, kijun_period, target_pnl_ratio, max_drowdown_ratio)
    print(res)

1.4479369687798221e-06
1.8636175971374832e-06
1.627279407828461e-06
2.4165821951448903e-06
3.6244284098147144e-06


In [22]:
class Objective:
    def __init__(self, start_ind, end_ind, selected_index_names):
        self.ire = IndexRuleEvaluator()
        self.start_ind = start_ind
        self.end_ind = end_ind
        self.selected_index_names = selected_index_names
        #self.index_max_val = index_max_val
        #self.index_min_val = index_min_val

    def __call__(self, trial):
        index_kijun_max_val = []
        index_kijun_min_val = []
        for i in range(len(self.selected_index_names)):
            index_name = self.selected_index_names[i]
            max_per = trial.suggest_int(index_name, 5, 99)
            min_per = trial.suggest_int(index_name, 1, max_per)
            index_kijun_max_val.append(np.percentile(MarketData.all_index[index_name][self.start_ind : self.end_ind+1], max_per))
            index_kijun_min_val.append(np.percentile(MarketData.all_index[index_name][self.start_ind : self.end_ind+1], min_per))
        kijun_period = trial.suggest_int('kijun_period', 360, 10000)
        target_pnl_ratio = trial.suggest_float('target_pnl_ratio', 0.01, 0.05)
        max_drowdown_ratio = trial.suggest_float('max_drowdown_ratio', 0.01, 0.05)
        res = self.ire.evaluate(self.start_ind, self.end_ind, self.selected_index_names, index_kijun_max_val, index_kijun_min_val, kijun_period, target_pnl_ratio, max_drowdown_ratio)
        return res



start_ind = 1000
end_ind = int(len(MarketData.close) * 0.3)
rcg = RandomIndexCombinationGenerator()
selected_index_names = rcg.get_randome_index_combinations()
objective = Objective(start_ind, end_ind, selected_index_names)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3000)
best_params = study.best_params
best_value = study.best_value
print("Best parameters:", best_params)
print("Best value:", best_value)


[I 2024-01-23 23:53:32,948] A new study created in memory with name: no-name-1ea323fd-02cb-4c77-9f0f-ed9202b123b8
/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:682: RuntimeWarning: Inconsistent parameter values for distribution with name "rsi_60"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 5, 'high': 99}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:682: RuntimeWarning: Inconsistent parameter values for distribution with name "dx_5"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these

Best parameters: {'rocr_15': 55, 'ma_kairi_720': 95, 'adx_720': 20, 'adx_15': 86, 'rsi_120': 21, 'cci_240': 43, 'dx_60': 67, 'ma_kairi_240': 55, 'rocr_240': 21, 'adx_60': 22, 'rsi_60': 40, 'rocr_5': 93, 'cmo_15': 92, 'cci_120': 98, 'correl_5': 28, 'minus_di_240': 73, 'dx_120': 33, 'rsi_15': 28, 'dx_5': 11, 'ma_kairi_5': 91, 'rsi_720': 39, 'rsi_5': 42, 'cci_15': 67, 'adx_240': 95, 'adx_5': 14, 'correl_240': 43, 'ma_kairi_15': 40, 'minus_di_60': 37, 'cmo_5': 84, 'rocr_60': 69, 'cmo_120': 26, 'cci_5': 76, 'minus_di_120': 7, 'cmo_720': 19, 'cci_60': 69, 'cci_720': 7, 'dx_720': 83, 'cmo_240': 70, 'adx_120': 61, 'rocr_120': 14, 'kijun_period': 6586, 'target_pnl_ratio': 0.03409108485004223, 'max_drowdown_ratio': 0.043292336687452755}
Best value: 0.0


In [ ]:
MarketData.high[1200000]

20356.0

In [ ]:
MarketData.high[1200749]

20356.0